In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import warnings,sys, json, itertools
warnings.filterwarnings('ignore')
from tqdm import tqdm
from pprint import pprint
import shapely.ops
import geopandas as gpd
from concurrent.futures import ThreadPoolExecutor, as_completed
import networkx as nx

India's assembly level data

with open('shapefiles-master/india/assembly/india_2012-17_AC.json') as f:
    data = json.load(f)
    pprint(data.keys())
    pprint(data['features'][0].keys())
    pprint(data['features'][0]['properties'])
    pprint(data['features'][0]['geometry'].keys())
    # pprint(data['features'][0]['geometry']['type'])
    # pprint(data['features'][0]['geometry']['coordinates'])
    # pprint(data['features'][0]['geometry']['coordinates'][0])

In [2]:
talukas_gdf = gpd.read_file('india-master/taluk/india_taluk.geojson')

In [3]:
display(talukas_gdf.head().T)

,0,1,2,3,4
ID_0,105,105,105,105,105
ISO,IND,IND,IND,IND,IND
NAME_0,India,India,India,India,India
ID_1,7,7,7,7,7
NAME_1,Chhattisgarh,Chhattisgarh,Chhattisgarh,Chhattisgarh,Chhattisgarh
ID_2,110,110,111,111,112
NAME_2,Kawardha,Kawardha,Korba,Korba,Koriya
ID_3,334,335,336,337,338
NAME_3,Kawardha,Pandaria,Katghora,Korba,Baikunthpur
NL_NAME_3,None,None,None,None,None


In [2]:
city_gdf = gpd.read_file('shapefiles-master/india/city/india_cities.json')
# city_gdf = city_gdf[city_gdf["state"]=="Karnataka"]
city_gdf = city_gdf.to_crs(epsg=3857)

In [3]:
# for state in sorted(city_gdf["state"].unique()):
#     print(state)
#     for city in sorted(city_gdf[city_gdf["state"]==state]["city"].values):
#         print("\t",city)

In [4]:
city_1 = "Bangalore"
state_1 = "Karnataka"
city_2 = "Mumbai"
state_2 = "Maharashtra"

In [5]:
city_info_dict = city_gdf[(city_gdf["city"]==city_1) | (city_gdf["city"]==city_2)].drop(columns=["cartodb_id","geometry"]).set_index("city").to_dict(orient="index")
del city_gdf
pprint(city_info_dict,indent=4)

{   'Bangalore': {'lat': 12.96199, 'lon': 77.590416, 'state': 'Karnataka'},
    'Mumbai': {'lat': 19.068493, 'lon': 72.880505, 'state': 'Maharashtra'}}


In [6]:
def AC_finder(gdf,point):
    AC=False
    for index, row in gdf.iterrows():
        if row['geometry'].contains(point):
                AC = {
                     "AC_NAME" : row["AC_NAME"],
                     "index" : index}
                break
        # else:
            # print("Point does not lie within any feature.")
    return AC


In [17]:
gdf = gpd.read_file('shapefiles-master/india/assembly/india_2012-17_AC.json').dropna().reset_index(drop=True)
print(gdf.shape[0])
# gdf = gdf[gdf["ST_NAME"]=="KARNATAKA"].reset_index(drop=True)
# print(gdf.shape[0])

4077


In [18]:
from shapely.geometry import Point
for city in city_info_dict:
    point = Point(city_info_dict[city]["lon"], city_info_dict[city]["lat"])
    # city_info_dict["Point"] = point
    state_gdf = gdf[gdf["ST_NAME"].str.upper()==city_info_dict[city]["state"].upper()]
    temp_gdf = state_gdf[state_gdf["DIST_NAME"].str.upper().str.contains(city.upper())]
    if temp_gdf.shape[0]==0:
        temp_gdf = state_gdf
    return_value = AC_finder(temp_gdf,point)
    if return_value:
        city_info_dict[city] = {**city_info_dict[city], **return_value}
        
    else:
        print("City not found in any ACs")
        sys.exit()
pprint(city_info_dict)
del temp_gdf, state_gdf

{'Bangalore': {'AC_NAME': 'Chickpet',
               'index': 1448,
               'lat': 12.96199,
               'lon': 77.590416,
               'state': 'Karnataka'},
 'Mumbai': {'AC_NAME': 'Kalina',
            'index': 2031,
            'lat': 19.068493,
            'lon': 72.880505,
            'state': 'Maharashtra'}}


In [19]:
projected_centroids = gdf.to_crs(epsg=3857).centroid
gdf["centroid"] = projected_centroids
gdf = gdf.to_crs(epsg=4326)
# gdf.plot()
# display(gdf.head(2).T)

In [20]:
needed_cols = ['ST_NAME', 'DIST_NAME', 'AC_NAME', 'PC_NAME', 'centroid',"geometry"]
gdf_dict = gdf[needed_cols].to_dict(orient="index")
gdf_shape = gdf.shape[0]
del gdf

In [21]:
# %%script echo skipping

import shapely.ops
import random

# Initialize an empty list to store adjacent pairs
adjacent_pairs_dict = {}
count = 0
centroid_of_city_1 = projected_centroids.iloc[city_info_dict[city_1]["index"]]
# print(AC_index_of_city_1)
# sys.exit()
# Iterate through each pair of features
for i in tqdm(gdf_dict):
    centroid_of_i = gdf_dict[i]["centroid"]
    i_dict = gdf_dict[i]
    distance = centroid_of_i.distance(centroid_of_city_1)
    i_dict["st_line_distance_to_city_1"] = round(distance/1000)
    #Assuming that the distance is the same for rail is between 1x and 1.42x for rail and road 1.42x to 2x
    i_dict["rail_distance_to_city_1"] = round((distance * random.uniform(1, 1.42))/1000)
    i_dict["road_distance_to_city_1"] = round((distance* random.uniform(1.42, 2))/1000)
    i_dict["Adjacent_to"]={}
    
    for j in range(i + 1, gdf_shape):
        j_dict = gdf_dict[j]
        try:
            if i_dict["geometry"].boundary.intersects(j_dict["geometry"].boundary):
                # Check if they actually share a part of their boundary
                if i_dict["geometry"].intersection(j_dict["geometry"].boundary).length > 0:
                    
                    # distance = round(projected_centroids.iloc[i].distance(projected_centroids.iloc[j])/1000)
                    distance = centroid_of_i.distance(j_dict["centroid"])
                    i_dict["Adjacent_to"][j_dict["AC_NAME"]] = {
                        "st_line_distance" : round(distance/1000),
                        "rail_distance" : round((distance * random.uniform(1, 1.42))/1000),
                        "road_distance" : round((distance* random.uniform(1.42, 2))/1000)
                    }
                    # adjacent_pairs.append((gdf.AC_NAME[i], gdf.AC_NAME[j],distance))

        except AttributeError:
            pass
    # del i_dict["centroid"]
    adjacent_pairs_dict[i] = i_dict
pprint(adjacent_pairs_dict[0])

100%|██████████| 4077/4077 [03:00<00:00, 22.54it/s] 

{'AC_NAME': 'Ichchapuram',
 'Adjacent_to': {'CHIKITI': {'rail_distance': 20,
                             'road_distance': 27,
                             'st_line_distance': 16},
                 'Palasa': {'rail_distance': 31,
                            'road_distance': 54,
                            'st_line_distance': 30}},
 'DIST_NAME': 'SRIKAKULAM',
 'PC_NAME': 'SRIKAKULAM',
 'ST_NAME': 'ANDHRA PRADESH',
 'centroid': <POINT (9418389.054 2155016.127)>,
 'geometry': <POLYGON ((84.765 19.077, 84.684 18.968, 84.604 18.89, 84.576 18.842, 84.527...>,
 'rail_distance_to_city_1': 1183,
 'road_distance_to_city_1': 1631,
 'st_line_distance_to_city_1': 1050}


In [22]:
# %%script echo skipping

import shapely.ops
import random

# Initialize an empty list to store adjacent pairs
adjacent_pairs_dict = {}
count = 0
centroid_of_city_1 = projected_centroids.iloc[city_info_dict[city_1]["index"]]
# print(AC_index_of_city_1)
# sys.exit()
# Iterate through each pair of features
for i in tqdm(gdf_dict):
    centroid_of_i = gdf_dict[i]["centroid"]
    i_dict = gdf_dict[i]
    distance = centroid_of_i.distance(centroid_of_city_1)
    i_dict["st_line_distance_to_city_1"] = round(distance/1000)
    #Assuming that the distance is the same for rail is between 1x and 1.42x for rail and road 1.42x to 2x
    i_dict["rail_distance_to_city_1"] = round((distance * random.uniform(1, 1.42))/1000)
    i_dict["road_distance_to_city_1"] = round((distance* random.uniform(1.42, 2))/1000)
    i_dict["Adjacent_to"]={}
    
    for j in range(i + 1, gdf_shape):
        j_dict = gdf_dict[j]
        try:
            if i_dict["geometry"].boundary.intersects(j_dict["geometry"].boundary):
                # Check if they actually share a part of their boundary
                if i_dict["geometry"].intersection(j_dict["geometry"].boundary).length > 0:
                    
                    # distance = round(projected_centroids.iloc[i].distance(projected_centroids.iloc[j])/1000)
                    distance = centroid_of_i.distance(j_dict["centroid"])
                    i_dict["Adjacent_to"][j_dict["AC_NAME"]] = {
                        "st_line_distance" : round(distance/1000),
                        "rail_distance" : round((distance * random.uniform(1, 1.42))/1000),
                        "road_distance" : round((distance* random.uniform(1.42, 2))/1000)
                    }
                    # adjacent_pairs.append((gdf.AC_NAME[i], gdf.AC_NAME[j],distance))

        except AttributeError:
            pass
    # del i_dict["centroid"]
    adjacent_pairs_dict[i] = i_dict
pprint(adjacent_pairs_dict[0])

100%|██████████| 4077/4077 [02:59<00:00, 22.71it/s] 

{'AC_NAME': 'Ichchapuram',
 'Adjacent_to': {'CHIKITI': {'rail_distance': 17,
                             'road_distance': 28,
                             'st_line_distance': 16},
                 'Palasa': {'rail_distance': 36,
                            'road_distance': 49,
                            'st_line_distance': 30}},
 'DIST_NAME': 'SRIKAKULAM',
 'PC_NAME': 'SRIKAKULAM',
 'ST_NAME': 'ANDHRA PRADESH',
 'centroid': <POINT (9418389.054 2155016.127)>,
 'geometry': <POLYGON ((84.765 19.077, 84.684 18.968, 84.604 18.89, 84.576 18.842, 84.527...>,
 'rail_distance_to_city_1': 1187,
 'road_distance_to_city_1': 1928,
 'st_line_distance_to_city_1': 1050}


In [23]:
# Assuming adjacent_pairs_dict is already defined and contains valid data
adjacent_pairs_json = json.dumps(adjacent_pairs_dict, indent=4)

# Write the JSON string to a file
with open('adjacent_pairs_dict.json', 'w') as f:
    f.write(adjacent_pairs_json)
sys.exit()

TypeError: Object of type Point is not JSON serializable

In [ ]:
!pip install pyrosm

In [ ]:
import osmnx as ox

# Define the origin and destination points
origin = (37.7749, -122.4194)  # San Francisco
destination = (34.0522, -118.2437)  # Los Angeles

# Get the driving route
route = ox.shortest_path(ox.graph_from_place("California, USA", network_type="drive"), origin, destination, weight='length')

# Calculate the driving distance
distance = sum(ox.utils_graph.get_route_edge_attributes(ox.graph_from_place("California, USA", network_type="drive"), route, 'length'))

print(f"Driving Distance: {distance} meters")

In [ ]:
temp_gdf = gpd.GeoDataFrame(pd.DataFrame.from_dict(adjacent_pairs_dict, orient='index'),
                            geometry='geometry')
temp_gdf.crs = "EPSG:4326"
display(temp_gdf.shape)
display(temp_gdf.head(2).T)
temp_gdf.to_file("adjacent_pairs_dict.geojson", driver="GeoJSON")

In [ ]:
sys.exit()

In [ ]:
# Assuming adjacent_pairs_dict is already defined and contains valid data
adjacent_pairs_json = json.dumps(adjacent_pairs_dict, indent=4)

# Write the JSON string to a file
with open('adjacent_pairs_dict.json', 'w') as f:
    f.write(adjacent_pairs_json)

Finding Assembly constituencies that are next to each other

In [ ]:
adjacent_pairs_dict

In [ ]:
adjacent_pairs_dict = json.load(open('adjacent_pairs_dict.json'))
# pprint(adjacent_pairs_dict)

In [ ]:
import networkx as nx
G = nx.Graph(heuristic=np.inf)
for key in tqdm(adjacent_pairs_dict):
    if adjacent_pairs_dict[key] == {}:
        continue
    if not(G.has_node(key)):
        G.add_node(key,heuristic=distance_dict[key])
    for neighbor in adjacent_pairs_dict[key]:
        if not(G.has_node(neighbor)):
            G.add_node(neighbor)
        G.add_edge(key, neighbor, weight=adjacent_pairs_dict[key][neighbor])

In [ ]:
%%time

import plotly.graph_objects as go
pos = nx.spring_layout(G)
node_x = [pos[node][0] for node in G.nodes()]
node_y = [pos[node][1] for node in G.nodes()]
edge_x = []
edge_y = []
for edge in G.edges():
    source, target = edge
    x0, y0 = pos[source]
    x1, y1 = pos[target]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

# Create edge labels (weights)
edge_labels = {edge: G.get_edge_data(*edge)['weight'] for edge in G.edges()}

fig = go.Figure(data=[go.Scatter(
    x=edge_x,
    y=edge_y,
    mode='lines',
    line_shape='spline',
    opacity=0.5,
    hoverinfo='none'
),
go.Scatter(
    x=node_x,
    y=node_y,
    mode='markers',
    hoverinfo='text',
    hovertext=[f"Node: {node}" for node in G.nodes()],
    marker=dict(size=10)
)])

# Add edge labels
for edge, label in edge_labels.items():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    x = (x0 + x1) / 2
    y = (y0 + y1) / 2
    fig.add_annotation(x=x, y=y, text=str(label), showarrow=False)

fig.update_layout(
    showlegend=False,
    hovermode='x',
    margin=dict(b=20, l=5, r=5, t=40),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))

fig.show()

In [ ]:
temp_gdf = gdf[gdf["ST_NAME"].str.upper()==city_info_dict[city]["state"].upper()]
display(temp_gdf.sort_values("DIST_NAME"))

In [ ]:

for index, row in gdf.iterrows():
    for city in city_info_dict:
    
        if row['geometry'].contains(point):  # Check if the point lies within the geometry
            print(f"Point lies in feature with index {index} and properties: {row}")
            break
    else:
        print("Point does not lie within any feature.")

In [ ]:
gdf.head()

In [ ]:
pprint(G.nodes.data())
    #   G.nodes["UDUPI CHIKMAGALUR"])

In [ ]:
shortest_path = nx.shortest_path(G, source="Bangalore South", target="Hubli-Dharwad-Central", weight='weight')
print("Shortest Path:", shortest_path)

In [ ]:
pprint(G.node_dict_factory("Bangalore South"))

In [ ]:
# Get the edge weights
edge_weights = nx.get_edge_attributes(G, 'weight')

# Position the nodes
pos = nx.spring_layout(G)

# Draw the graph
nx.draw(G, pos, with_labels=True)

# Draw edge labels (weights)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_weights)

plt.show()

In [ ]:
gdf_dicty = {gdf.iloc[0]["AC_NAME"] : {"Centroid" : }for i in gdf }
display(gdf_adj.head().T)

In [ ]:
G = nx.Graph()
G.add_nodes_from(range(len(gdf)))
# G.add_edges_from(adjacent_pairs)
nx.draw(G)
# nx.draw(G, with_labels=True)
plt.show()